In [1]:
#Importar las librerias a utilizar
import numpy as np
import pandas as pd
import re
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

#Link con los datos
data_source_url = "https://raw.githubusercontent.com/mhemmg/datasets/master/nlp/airline_tweets.csv"

In [2]:
#Función para limpiar el texto de links, signos especiales y dobles espacios.
punct = "[!#$%&()*+,-./:;<=>?@[\]^_`{|}~]"
def clean_text(text):
    text = text.lower()
    new_text = re.sub('https:[^,\s]+', ' ', text)
    new_text = re.sub('(rt|@|:)+', ' ', new_text)
    new_text = re.sub(punct, ' ', new_text)
    new_text = re.sub('  +', ' ', new_text)
    return new_text

In [3]:
#Preprocesamiento de los datos, se limpian y se categorizan los sentimientos
def preprocess(data_source_url, clean_func):
    temp = pd.read_csv(data_source_url)
    features_labels = temp.loc[:,('text','airline_sentiment')]
    features_labels['airline_sentiment_bin'] = features_labels.airline_sentiment.map({'neutral': 0, 'positive': 1,'negative':2})
    features_labels['text_clean'] = features_labels['text'].apply(lambda t: clean_func(t))
    airline_tweets = features_labels[['text_clean','airline_sentiment_bin']]
    return airline_tweets

In [4]:
#Tabla de datos final
airline_tweets = preprocess(data_source_url, clean_text)
airline_tweets.head()

,text_clean,airline_sentiment_bin
0,virginamerica what dhepburn said,0
1,virginamerica plus you've added commercials t...,1
2,virginamerica i didn't today must mean i need...,0
3,virginamerica it's really aggressive to blast...,2
4,virginamerica and it's a really big bad thing...,2


In [5]:
#Entrenamiento y testeo con random_state fijo para TextBlob y Azure
X_train, X_test, y_train, y_test = train_test_split(airline_tweets['text_clean'], airline_tweets['airline_sentiment_bin'], test_size=0.2, random_state=1)

## TextBlob

En esta sección se utiliza la librería textblob para hacer análisis de sentimientos como se aprendió en clase

In [6]:
#Analisis de sentimientos con TextBlob aprendido en clase
textblob_test = []

#Librería
from textblob import TextBlob

for i in X_test:
    sentiment = TextBlob(i).sentiment.polarity
    if sentiment < 0:
        textblob_test.append(2)
    elif sentiment == 0:
        textblob_test.append(0)
    else:
        textblob_test.append(1)

In [7]:
#Precisión del análisis
tbscore = accuracy_score(textblob_test,y_test)
print(tbscore)

0.46209016393442626


## Microsoft Azure

En esta sección se utiliza el análisis de textos que provee microsoft azure para analizar los sentimientos de los datos usados

In [8]:
#key y endpoint de mi cuenta de azure
key = "0075b6399ea34cb29e92396d97230928"
endpoint = "https://manu0221.cognitiveservices.azure.com/"

#Librerias necesarias
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

#Autenticación del cliente como dice en la documentación
def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

In [9]:
#Analisi de sentimientos con azure siguiendo indicaciones de la documentación
azure_test = []
X_test_new = []

for i in X_test:
    X_test_new.append(i)
result = []

for i in np.arange(0,len(X_test_new),10):
    result.append(client.analyze_sentiment(X_test_new[i:i+10]))

for doc_result in result:
    for document in doc_result:
        sentiment = document.sentences[0].sentiment
        if sentiment == 'positive':
            azure_test.append(1)
        if sentiment == 'neutral':
            azure_test.append(0)
        if sentiment == 'negative':
            azure_test.append(2)

In [10]:
#Precisión del análisis
mascore = accuracy_score(azure_test,y_test)
print(mascore)

0.617827868852459


## Sklearn

En esta sección se utilizan distintos métodos de sklearn para análizar los datos. Se utilizan los métodos con parámentros predeterminados para encontrar cuál es el que da mejor resultado, a este se le varían los parámetros para encontrar sus funciones óptimas para este problema

In [11]:
#Vectorización del texto de los datos y división en entrenamiento y testeo
#con mismo random_state al usado anteriormente
vectorizer = TfidfVectorizer()
X_vec = vectorizer.fit_transform(airline_tweets['text_clean']).toarray()
X_train, X_test, y_train, y_test = train_test_split(X_vec, airline_tweets['airline_sentiment_bin'], test_size=0.2, random_state=1)

In [12]:
#Métodos de clasificación a probar
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

In [13]:
knc = KNeighborsClassifier()
knc.fit(X_train,y_train)
kncscore = knc.score(X_test,y_test)
print(kncscore)

0.6970628415300546


In [14]:
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)
clfscore = clf.score(X_test,y_test)
print(clfscore)

0.667691256830601


In [15]:
rfc  = RandomForestClassifier()
rfc.fit(X_train,y_train)
rfcscore = rfc.score(X_test,y_test)
print(rfcscore)

0.7404371584699454


In [16]:
abc = AdaBoostClassifier()
abc.fit(X_train,y_train)
abcscore = abc.score(X_test,y_test)
print(abcscore)

0.725068306010929


In [17]:
gnb = GaussianNB()
gnb.fit(X_train,y_train)
gnbscore = gnb.score(X_test,y_test)
print(gnbscore)

0.5105874316939891


In [18]:
# #Variación de parámetros, esta celda es muy lenta, no se recomienda ejecutar
# model = RandomForestClassifier()
# grid = dict()
# grid['criterion'] = np.array(['gini','entropy','log_loss'])
# grid['max_features'] = np.array(['log2','sqrt',None])
# search = GridSearchCV(model,grid,scoring='neg_mean_absolute_error', n_jobs=None)
# results = search.fit(X_train, y_train)
# print('Config: %s' % results.best_params_)

## Comparación

In [19]:
data = [['TextBlob',tbscore*100],['Microsoft Azure',mascore*100], ['GaussianNB',gnbscore*100],['DecisionTreeClassifier',clfscore*100],['KNeighborsClassifier',kncscore*100],['AdaBoostClassifier',abcscore*100],['RandomForestClassifier',rfcscore*100]]

df = pd.DataFrame(data, columns=['Método', 'Precisión %'])

In [21]:
display(df)

,Método,Precisión %
0,TextBlob,46.209016
1,Microsoft Azure,61.782787
2,GaussianNB,51.058743
3,DecisionTreeClassifier,66.769126
4,KNeighborsClassifier,69.706284
5,AdaBoostClassifier,72.506831
6,RandomForestClassifier,74.043716


## Conclusiones

1. Los métodos pre-entrenados ofrecen una forma rápida de clasificar datos sin necesidad de poseer con un dataset previo de gran amplitud. Tanto TextBlob como la herramienta ofrecida por Microsoft Azure son buenas.

2. De tener que entrenar los modelos por cuenta propia es importante conoscer las estructuras de los métodos ofrecidos por sklearn y así deducir cuál se encaja mejor a los datos que se están manejando en el momento.

3. Los métodos de limpieza de datos pueden cambiar en gran medida os resultados obtenidos, es importante realizar estudios a mayor profundidad de esto.